## Meteo Bakery - Presentation Figures
In this notebook, we will generate figures for presentation

### import libraries

In [ ]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import meteo_utils as meteo

from itertools import product
from sklearn.preprocessing import MinMaxScaler

## Cross-Validation

### load data

In [ ]:
# load cross-validation results
scores_merged = pd.read_csv('../models/lgbm_optimized/cross_validation.csv')

### stack data to prepare for visualizations

In [ ]:
# stack and group according to model
scores_grouped = pd.DataFrame(scores_merged[['group', 'MAPE_mean_naive', 'MAPE_mean_lgbm_time', 'MAPE_mean_lgbm_weather']].set_index('group').stack().reset_index().iloc[:-3, :])
scores_grouped.columns = ['group', 'model', 'MAPE_mean']
scores_grouped['MAPE_std'] = pd.DataFrame(scores_merged[['group',  'MAPE_std_naive', 'MAPE_std_lgbm_time', 'MAPE_std_lgbm_weather']].set_index('group').stack().reset_index().iloc[:-3, :])[0]
scores_grouped['model'] = [x.split('_')[-1] for x in scores_grouped['model']]

# multiply with 100 to get MAPE scores in %
scores_grouped[['MAPE_mean', 'MAPE_std']] = scores_grouped[['MAPE_mean', 'MAPE_std']] *100

In [ ]:
# extract branch and product information as separate columns
scores_grouped['branch'] = [x.split(' | ')[0] for x in scores_grouped['group']]
scores_grouped['product'] = [x.split(' | ')[1] for x in scores_grouped['group']]
scores_grouped.head()

### plot mean MAPE and standard deviation from cross-validation over all groups

In [ ]:
fig = plt.figure(figsize=(3,3))
sns.barplot(data=scores_grouped, x='model', y='MAPE_mean', color='#2aa2cc', edgecolor='black', ci=None)
plt.ylabel('Average prediction error [%]', fontsize=12)
plt.yticks(np.arange(0, 26, 5), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 3), labels=['Baseline', 'LightGBM time', 'LightGBM weather'], fontsize=12, rotation=45, ha='right');
plt.title('Model Comparison', fontsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3,3))
sns.barplot(data=scores_grouped, x='model', y='MAPE_std', color='#2aa2cc', edgecolor='black', ci=None)
plt.ylabel('Variability of prediction error [%]', fontsize=12)
plt.yticks(np.arange(0, 26, 5), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 3), labels=['Baseline', 'LightGBM time', 'LightGBM weather'], fontsize=12, rotation=45, ha='right');
plt.title('Model Comparison', fontsize=14)
plt.show()

### plot scores separaly for each branch

In [ ]:
fig = plt.figure(figsize=(4,3))
sns.barplot(data=scores_grouped, x='branch', y='MAPE_mean', edgecolor='black', ci=None, hue='model', palette=['#d6633a', '#34831B', '#1b6883'],
                    order=['Metro', 'Train_Station', 'Center'])
plt.ylabel('Average prediction error [%]', fontsize=12)
plt.yticks(np.arange(0, 26, 5), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 3), labels=['Metro', 'Train Station', 'Center'], fontsize=12, rotation=45, ha='right');
plt.title('Model Comparison', fontsize=14)
leg= plt.legend(bbox_to_anchor=(1.01, 0.4), loc='upper left', frameon=False, fontsize=11)
leg.get_texts()[0].set_text('Baseline')
leg.get_texts()[1].set_text('LightGBM time')
leg.get_texts()[2].set_text('LightGBM weather')

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,3))
sns.barplot(data=scores_grouped, x='branch', y='MAPE_std', edgecolor='black', ci=None, hue='model', palette=['#d6633a', '#34831B', '#1b6883'],
                    order=['Metro', 'Train_Station', 'Center'])
plt.ylabel('Variability of prediction error [%]', fontsize=12)
plt.yticks(np.arange(0, 26, 5), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 3), labels=['Metro', 'Train Station', 'Center'], fontsize=12, rotation=45, ha='right');
plt.title('Model Comparison', fontsize=14)
leg= plt.legend(bbox_to_anchor=(1.01, 0.4), loc='upper left', frameon=False, fontsize=11)
leg.get_texts()[0].set_text('Baseline')
leg.get_texts()[1].set_text('LightGBM time')
leg.get_texts()[2].set_text('LightGBM weather')

plt.show()

## Feature Importance

### load data

In [ ]:
lgbm_fimportance_rel = pd.read_csv('../models/lgbm_optimized/rel_feature_importance.csv')

### stack data to prepare for visualizations

In [ ]:
# stack dataframe and group according to feature
lgbm_fi_stacked = lgbm_fimportance_rel.set_index('group').stack().reset_index()
lgbm_fi_stacked.columns = ['group', 'features', 'importance']
lgbm_fi_stacked = lgbm_fi_stacked[lgbm_fi_stacked['group']!='mean']

In [ ]:
# extract branch and product information as separate columns
lgbm_fi_stacked['branch'] = [x.split(' | ')[0] for x in lgbm_fi_stacked['group']]
lgbm_fi_stacked['product'] = [x.split(' | ')[1] for x in lgbm_fi_stacked['group']]

In [ ]:
lgbm_fi_stacked.head()

In [ ]:
# extract features ordered by the relative feature importance
col_order = lgbm_fimportance_rel.set_index('group').sort_values(by='mean', axis=1, ascending=False).columns
col_order

In [ ]:
# create feature map for renaming features during plotting
feature_map = {'turnover_lag_7':'turnover [lag 7]', 
                'day_of_week':'day of week', 
                'public_holiday': 'public holiday', 
                'turnover_lag_365': 'turnover [lag 365]',
                'temp_mean': 'temperature [daily mean]', 
                'snow_1h_mean_dev': 'snowfall [season. dev.]', 
                'month_cos': 'month [cosine-t.]', 
                'school_holiday': 'school holiday',
                'temp_mean_dev': 'temperature [season. dev.]', 
                'temp_mean_lead_1': 'temperature [next day]', 
                'month_sin': 'month [sine-t.]', 
                'rain_1h_mean_dev': 'rainfall [season. dev.]',
                'humidity_mean': 'humidity [daily mean]', 
                'pressure_mean_dev': 'atm. pressure [season. dev.]', 
                'humidity_mean_dev': 'humidity [season. dev.]',
                'pressure_mean_change': 'atm. pressure [change]', 
                'temp_mean_change': 'temperature [change]', 
                'humidity_mean_change': 'humidity [change]',
                'rain_1h_mean': 'rainfall [daily mean]', 
                'rain_1h_mean_lead_1': 'rainfall [next day]', 
                'day_hazy': 'hazy day', 
                'day_clear': 'clear day',
                'day_frosty': 'frosty day', 
                'day_summer': 'summer day', 
                'snow_1h_mean_lead_1': 'snowfall [next day]', 
                'snow_1h_mean': 'snowfall [daily mean]',
                'day_thunder': 'thunder day'
                }

### plot global feature importance averaged over all groups

In [ ]:
fig = plt.figure(figsize=(7, 6))
sns.barplot(data=lgbm_fi_stacked, y='features', x='importance', color='#2aa2cc', edgecolor='black', ci=None, order=col_order)
plt.xlabel('Relative Importance [%]', fontsize=12)
plt.xticks(ticks=np.arange(0, 51, 10), labels=np.arange(0, 51, 10), fontsize=11)
plt.ylabel(None)
plt.yticks(ticks=np.arange(0, 27), labels=col_order.map(feature_map), fontsize=10)
plt.title('Feature Importance', fontsize=14)
plt.show()


### plot top 6 features

In [ ]:
fig = plt.figure(figsize=(7, 3))
#fig.patch.set_visible(False)
sns.barplot(data=lgbm_fi_stacked[lgbm_fi_stacked['importance']>2], y='features', x='importance', color='#2aa2cc', 
                    edgecolor='black', ci=None, order=col_order[:6])
plt.xlabel('Relative Importance [%]', fontsize=13)
plt.xticks(ticks=np.arange(0, 52, 10), labels=np.arange(0, 52, 10), fontsize=11)
plt.ylabel(None)
plt.yticks(ticks=np.arange(0, 6), labels=col_order[:6].map(feature_map), fontsize=11)
plt.title('Feature Importance (Top 6)', fontsize=14)
plt.show()

### plot feature importance of weather features only separately for each product

In [ ]:
for product in lgbm_fi_stacked['product'].unique():
    fimp = lgbm_fimportance_rel.copy()
    fimp.drop(['turnover_lag_7', 'turnover_lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday'], 
                    axis=1, inplace=True)
    feature_cols = fimp.columns[1:]
    fimp.loc[:14, 'branch'] = [x.split(' | ')[0] for x in fimp.loc[:14, 'group']]
    fimp.loc[:14, 'product'] = [x.split(' | ')[1] for x in fimp.loc[:14, 'group']]
    fimp = fimp[fimp['product']==product]
    fimp.loc[3, 'group'] = 'mean'
    fimp.loc[3, feature_cols] = [np.mean(fimp[x]) for x in feature_cols]
    temp_order = fimp[fimp.columns[:-2]].set_index('group').sort_values(by='mean', axis=1, ascending=False).columns
    
    fig = plt.figure(figsize=(7, 5))
    fig.patch.set_visible(False)
    sns.barplot(data=lgbm_fi_stacked[lgbm_fi_stacked['product']==product], y='features', x='importance', 
                    color='#2aa2cc', edgecolor='black', ci=None, order=temp_order)
    plt.xlabel('Relative Importance [%]', fontsize=12)
    plt.xticks(ticks=np.arange(0, 6, 1), labels=np.arange(0, 6, 1), fontsize=11)
    plt.ylabel(None)
    plt.yticks(ticks=np.arange(0, 20), labels=temp_order.map(feature_map), fontsize=10)
    plt.title(f'Feature Importance - {product}', fontsize=14)
    plt.show()

## Get predictions, residuals, and feature contributions
Here we will examine predictions, residuals, and feature contributions as assessed through shap values for the LightGBM model trained with temporal and weather features over a whole year. We will also examine predictions and residuals for the naive baseline model for comparison.

### load data

In [ ]:
df = pd.read_csv('../data/data_final.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

### generate train and test df

In [ ]:
df_train = df[df.year<2018]
df_test = df[df.year>=2018]

In [ ]:
# define feature set
weather_features = ['turnover_lag_7', 'turnover_lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                                    'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean',
                                                    'day_frosty', 'day_thunder', 'day_clear','day_hazy', 'day_summer',
                                                    'temp_mean_dev', 'humidity_mean_dev', 'pressure_mean_dev', 'rain_1h_mean_dev', 'snow_1h_mean_dev',
                                                    'temp_mean_change', 'pressure_mean_change', 'humidity_mean_change',
                                                    'temp_mean_lead_1', 'rain_1h_mean_lead_1', 'snow_1h_mean_lead_1']

# define hyperparameters
params_optimal = {
    'boosting_type': 'dart',
    'n_estimators': 200,
    'learning_rate': 0.1
}

### get predictions, residuals, and shap values for a whole year

In [ ]:
model_preds = meteo.get_preds_and_shaps(df_train, grouping_vars=['branch', 'product'], target='turnover', features=weather_features,
                              lgbm_kwargs=params_optimal, splits=52, compute_baseline=True)

In [ ]:
# show and example predictions dataframe for a single time series for inspection
model_preds['predictions'][0]

### visualize predictions and residuals for individual time series

In [ ]:
# iterate over all time series and plot predictions and residuals
for i in range(len(model_preds['group'])):
    data_temp = model_preds['predictions'][i]
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 4))
    fig.suptitle(model_preds['group'][i])
    sns.lineplot(data=data_temp, x=data_temp.index, y='y_true', color='grey', 
                        label='observed', ax=ax1)
    sns.lineplot(data=data_temp, x=data_temp.index, y='y_pred_baseline', color='#d6633a',
                        label='predicted by Baseline', ax=ax1)
    sns.lineplot(data=data_temp, x=data_temp.index, y='y_pred_lgbm', color='#1b6883', 
                        label='predicted by LightGBM', ax=ax1)
    ax1.set_ylabel('Turnover [€]', fontsize=12)
    ax1.set_ylim(0, data_temp['y_true'].max()+100)
    ax1.set_xlabel(None)
    ax1.legend(bbox_to_anchor=(1.01, 0.4), loc='upper left', frameon=False, fontsize=10)

    sns.scatterplot(data=data_temp, x=data_temp.index, y='residual_baseline', 
                    color='#d6633a', edgecolor='black', label='Baseline', ax=ax2)
    sns.scatterplot(data=data_temp, x=data_temp.index, y='residual_lgbm', 
                    color='#1b6883', edgecolor='black', label='LightGBM', ax=ax2)
    ax2.set_ylabel('Model error [€]', fontsize=12)
    ax2.set_ylim((data_temp[['residual_lgbm', 'residual_baseline']].min().min())-100, 
                    (data_temp[['residual_lgbm', 'residual_baseline']].max().max())+100)
    ax2.set_xlabel(None)
    ax2.legend(bbox_to_anchor=(1.01, 0.4), loc='upper left', frameon=False, fontsize=10)

    plt.tight_layout()
    plt.show()

### sum up residuals to get an estimate of the yearly total error in €
We will sum up all individual errors (i.e. residuals) across a whole year to get the yearly summed error per branch/product combination. We will then sum up these errors over all branch/product combinations to get an estimate of the yearly total error in €. We will perform these calculations separately for positive residuals (i.e. errors due to overestimating sales) and negative residuals (i.e. errors due to underestimating sales).

In [ ]:
summed_error = pd.DataFrame({'group': [], 'pos_baseline': [], 'neg_baseline': [], 'pos_lgbm': [], 'neg_lgbm': []})
for i in range(len(model_preds['group'])):
    data_temp = model_preds['predictions'][i]
    sum_pos_baseline = data_temp[data_temp['residual_baseline']>=0]['residual_baseline'].sum() 
    sum_neg_baseline = data_temp[data_temp['residual_baseline']<0]['residual_baseline'].sum() 
    sum_pos_lgbm = data_temp[data_temp['residual_lgbm']>=0]['residual_lgbm'].sum() 
    sum_neg_lgbm = data_temp[data_temp['residual_lgbm']<0]['residual_lgbm'].sum() 

    summed_error.loc[i, 'group'] = model_preds['group'][i]
    summed_error.loc[i, 'pos_baseline'] = sum_pos_baseline
    summed_error.loc[i, 'neg_baseline'] = np.abs(sum_neg_baseline)
    summed_error.loc[i, 'pos_lgbm'] = sum_pos_lgbm
    summed_error.loc[i, 'neg_lgbm'] = np.abs(sum_neg_lgbm)

In [ ]:
# sum over all branch/product combinations
summed_error.loc[15, 'group'] = 'sum'
summed_error.loc[15, summed_error.columns[1:]] = [np.sum(summed_error[x]) for x in summed_error.columns[1:]]
summed_error

### stack data to prepare for visualizations

In [ ]:
# group summed errors according to error type and model
summed_error_grouped = summed_error.set_index('group').stack().reset_index()
summed_error_grouped.columns=['group', 'error_type', 'error']
summed_error_grouped['model'] = [x.split('_')[1] for x in summed_error_grouped['error_type']]
summed_error_grouped['error_type'] = [x.split('_')[0] for x in summed_error_grouped['error_type']]
summed_error_grouped['error_type'].replace('pos', 'overestimation', inplace=True)
summed_error_grouped['error_type'].replace('neg', 'underestimation', inplace=True)
summed_error_grouped.tail()

### plot total yearly summed error

In [ ]:
fig = plt.figure(figsize=(4,3.5))
sns.barplot(data=summed_error_grouped[(summed_error_grouped['group']=='sum')].groupby('model').sum().reset_index(),
                x='model', y='error', color='#2aa2cc', edgecolor='black', errwidth=0)
plt.ylabel('Yearly summed error [€]', fontsize=12)
plt.yticks(ticks=np.arange(0, 400001, 100000), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 2), labels=['Baseline', 'LightGBM weather'], rotation=45, ha='right',  fontsize=12)
plt.title('Financial loss', fontsize=14)

plt.show()

### plot yearly summed error due to both over- and underestimation

In [ ]:
# color-code according to errors due to over- and underestimation
fig = plt.figure(figsize=(4,3.5))
sns.barplot(data=summed_error_grouped[(summed_error_grouped['group']=='sum')].groupby('model').sum().reset_index(),
                x='model', y='error', color='#1b6883', edgecolor='black', errwidth=0, label='Overestimation')
plt.ylabel('Yearly summed error [€]', fontsize=12)
sns.barplot(data=summed_error_grouped[(summed_error_grouped['group']=='sum') & (summed_error_grouped['error_type']=='underestimation')],
                x='model', y='error', color='#d6633a', edgecolor='black', errwidth=0, label='Underestimation')
plt.ylabel('Yearly summed error [€]', fontsize=12)
plt.yticks(ticks=np.arange(0, 400001, 100000), fontsize=11)
plt.xlabel(None)
plt.xticks(ticks=np.arange(0, 2), labels=['Baseline', 'LightGBM weather'], rotation=45, ha='right', fontsize=12)
plt.title('Financial loss', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 0.2), loc='upper left', frameon=False, fontsize=11)

plt.show()

### plot shap values from cross-validation

In [ ]:
# extract example shap values for brief inspection
model_preds['shap_lgbm'][0].head()

### make shap plot for an individual time series

In [ ]:
for i in range(len(model_preds['group'])):
    # extract shap values for individual time series and stack according to features
    shap_temp = model_preds['shap_lgbm'][i][weather_features].stack().reset_index()
    # drop unused columns created by resetting index and rename
    shap_temp.drop(columns=['level_0'], inplace=True)
    shap_temp.columns = ['features', 'shap']

    # extract feature values for individual time series and min-max scale; fill NaN w/ 1 before stacking, otherwise such rows will be dropped!
    scaler = MinMaxScaler()
    features_scaled = pd.DataFrame(scaler.fit_transform(model_preds['features_lgbm'][i][weather_features].fillna(1)), columns=weather_features)
    # stack features and append last column to temporary shap df
    shap_temp['X'] = features_scaled.stack().reset_index().iloc[:, -1]

    # get columns ranked by mean absolute shap values and extract sorted column names
    shap_order = model_preds['shap_lgbm'][i][weather_features].abs().mean().sort_values(ascending=False).index
    # re-order temporary shap df accordingly
    shap_temp['features'] = shap_temp['features'].astype('category')
    shap_temp['features'].cat.reorder_categories(shap_order, inplace=True)

    # plot shap values per branch/product combination
    fig, ax = plt.subplots(figsize=(7, 5))
    sns.scatterplot(data=shap_temp, y='features', x='shap', hue='X', palette='RdBu_r', edgecolor=None, alpha=0.5, ax=ax)

    sm = plt.cm.ScalarMappable(cmap="RdBu_r")
    sm.set_array([])
    # Remove the legend and add a colorbar
    ax.get_legend().remove()
    ax.figure.colorbar(sm)

    ax.set_xlim(shap_temp['shap'].min()-10, shap_temp['shap'].max()+10)
    ax.set_xlabel('Shap value', fontsize=12)
    ax.set_ylabel(None)
    ax.set_yticks(ticks=np.arange(0, 27))
    ax.set_yticklabels(labels=shap_order.map(feature_map), fontsize=10)

    plt.title(model_preds['group'][i])
    plt.show()
    

## Predictions on test set
Here, we will generate predictions for the test set for defined time window (restricted to 7 days). We will use the first January week in 2019 for illustration. Note that LightGBM reaches stable performance even if it was only trained with data up to 2017.

In [ ]:
test_pred = meteo.LGBM_predict(df_train, df_test, grouping_vars=['branch', 'product'], target='turnover', features=weather_features, 
                    lgbm_kwargs=params_optimal, start_date='2019-01-01', end_date='2019-01-07', compute_shap=True, plot=True, show_baseline=True)